# Denoising Diffusion Probabilistic Models with M&Ms-2 Dataset

This tutorial illustrates how to use MONAI for training a denoising diffusion probabilistic model (DDPM)[1] to create
synthetic 2D images.

[1] - Ho et al. "Denoising Diffusion Probabilistic Models" https://arxiv.org/abs/2006.11239


## Setup environment

In [ ]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import time
import torchio
import torchmetrics

from monai import transforms
from monai import handlers
from monai.apps import MedNISTDataset
from monai.config import print_config
from monai.data import CacheDataset, DataLoader
from monai.utils import first, set_determinism

from generative.inferers import DiffusionInferer
from generative.networks.nets import DiffusionModelUNet
from generative.networks.schedulers import DDIMScheduler, DDPMScheduler, PNDMScheduler

print_config()
print("TorchIO version:", torchio.__version__)
print("TorchMetrics version:", torchmetrics.__version__)

### Metrics testing block: IS and FID
IS: https://torchmetrics.readthedocs.io/en/stable/image/inception_score.html  
FID: https://torchmetrics.readthedocs.io/en/stable/image/frechet_inception_distance.html

In [1]:
import torch
_ = torch.manual_seed(123)
from torchmetrics.image.inception import InceptionScore
inception = InceptionScore()
# generate some images
imgs = torch.randint(0, 255, (100, 3, 299, 299), dtype=torch.uint8)
inception.update(imgs)
inception.compute()

c:\Users\jrimm\.conda\envs\ddpm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jrimm\.conda\envs\ddpm\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


(tensor(1.0544), tensor(0.0117))

In [2]:
import torch
_ = torch.manual_seed(123)
from torchmetrics.image.fid import FrechetInceptionDistance
fid = FrechetInceptionDistance(feature=64)
# generate two slightly overlapping image intensity distributions
imgs_dist1 = torch.randint(0, 200, (100, 3, 299, 299), dtype=torch.uint8)
imgs_dist2 = torch.randint(100, 255, (100, 3, 299, 299), dtype=torch.uint8)
fid.update(imgs_dist1, real=True)
fid.update(imgs_dist2, real=False)
fid.compute()

tensor(12.7202)

## Setup data directory

You can specify a directory with the MONAI_DATA_DIRECTORY environment variable.

This allows you to save results and reuse downloads.

If not specified a temporary directory will be used.

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

## Set deterministic training for reproducibility

In [ ]:
set_determinism(42)

## Setup M&Ms-2 Dataset and training and validation dataloaders
Combine Sina's implementation to import M&Ms-2 dataset (https://www.ub.edu/mnms-2/) manually.

In [ ]:
from modules.dataloader import CMRCineDataModule
import argparse
import sys
sys.argv=['']
del sys
import os
import warnings
warnings.filterwarnings('ignore')

PATH = os.path.join(os.getcwd(), "dataset_3D_crop_small")
RUN_NAME = "cmr_DDPM_24042023"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print('Device is {}'.format(DEVICE))
# print("Current device is on", torch.cuda.current_device())
# print(torch.cuda.get_device_name(0))

# set your own path here, eg, '/home/bme001/20180883/data/mnms2/sorted/SA/PerDisease' (Linux style path)
default_config = {
    'dataset_path': PATH,
    'run_name': RUN_NAME,
    'epochs': 300,
    'log_interval': 100,
    'batch_size' : 8,
    'image_size' : 64,
    # TODO: num_workers > 0 causes error on windows
    'num_workers' : 0,  # default 8, windows cannot handle this
    'device' : DEVICE,
    'lr' : 3e-4,
    'noise_steps' : 1000,
    'beta_start': 1e-4,
    'beta_end': 0.02,
    }

# set key-value pairs from command line 
parser = argparse.ArgumentParser()
for keys in default_config:
    parser.add_argument('--'+keys, default=default_config[keys], type= type(default_config[keys]))
args = parser.parse_args()

# CMR2DDataModule or CMRCineDataModule class
data = CMRCineDataModule(
        data_dir = args.dataset_path,
        image_size = args.image_size,
        batch_size = args.batch_size,
        train_val_ratio = 0.8,
        num_workers = args.num_workers,
    )
data.prepare_data()
data.setup()

train_loader = data.train_dataloader()
val_loader = data.val_dataloader()

print(train_loader.dataset.__len__())
print(val_loader.dataset.__len__())

In [ ]:
print('Batch_size = {}'.format(args.batch_size))
print('Number of batches in train_dataloader = {}'.format(len(train_loader)))
print('Number of batches in val_dataloader = {}'.format(len(val_loader)))

batch = next(iter(train_loader))

print(batch['image']['data'].squeeze(-1).shape)
print(batch['image']['data'].squeeze(-1).max())
print(batch['image']['data'].squeeze(-1).min())

In [ ]:
from modules.utils import plot_batch

# TODO: Filter out the useless slices
plot_batch(train_loader)

### Visualisation of the training images

In [ ]:
check_data = first(train_loader)
print(f"batch shape: {check_data['image']['data'].shape}")
# print(check_data["image"]['data'][0, 0].shape)
image_visualisation = torch.cat(
    [check_data["image"]['data'][0, 0], check_data["image"]['data'][1, 0], \
        check_data["image"]['data'][2, 0], check_data["image"]['data'][3, 0]], dim=1
)
print(check_data["image"]['data'][0, 0].max())
print(check_data["image"]['data'][0, 0].min())
plt.figure("training images", (12, 6))
plt.imshow(image_visualisation, vmin=-1, vmax=1, cmap="gray")  
# plt.imshow(image_visualisation, cmap="gray")
plt.axis("off")
plt.tight_layout()
plt.show()

### Define network, scheduler, optimizer, and inferer
At this step, we instantiate the MONAI components to create a DDPM, the UNET, the noise scheduler, and the inferer used for training and sampling. We are using
the original DDPM scheduler containing 1000 timesteps in its Markov chain, and a 2D UNET with attention mechanisms
in the 2nd and 3rd levels, each with 1 attention head.

In [ ]:
# set up the model
device = torch.device(args.device)
total_timesteps = args.noise_steps
epochs = args.epochs

model = DiffusionModelUNet(
    spatial_dims = 2,
    in_channels = 1,
    out_channels = 1,
    num_channels = (128, 256, 256),
    attention_levels = (False, True, True),
    num_res_blocks = 1,
    num_head_channels = 256,
)
model.to(device)

scheduler = DDPMScheduler(num_train_timesteps=total_timesteps)

optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr)

inferer = DiffusionInferer(scheduler)

In [ ]:
assert False

## Model training
If you would like to skip the training and use a pre-trained model instead, set `use_pretrained=True`. This model was trained using the code in `tutorials/generative/distributed_training/ddpm_training_ddp.py`

In [ ]:
use_pretrained = False  # set to False to train the model from scratch

# TODO: Solve the problem CPU bottleneck, eg. CacheDataset https://www.youtube.com/watch?v=0mVgu_DFbeY
if use_pretrained:
    model = torch.hub.load("marksgraham/pretrained_generative_models:v0.2", model="ddpm_2d", verbose=True).to(device)
else:
    n_epochs = epochs
    val_interval = 30
    epoch_loss_list = []
    val_epoch_loss_list = []

    scaler = GradScaler()
    total_start = time.time()
    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), ncols=70)
        progress_bar.set_description(f"Epoch {epoch}")
        for step, batch in progress_bar:
            images = batch["image"]['data'].squeeze(-1).to(device)
            optimizer.zero_grad(set_to_none=True)

            with autocast(enabled=True):
                # Generate random noise
                noise = torch.randn_like(images).to(device)

                # Create timesteps
                timesteps = torch.randint(
                    0, inferer.scheduler.num_train_timesteps, (images.shape[0],), device=images.device
                ).long()

                # Get model prediction
                noise_pred = inferer(inputs=images, diffusion_model=model, noise=noise, timesteps=timesteps)

                loss = F.mse_loss(noise_pred.float(), noise.float())

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()

            progress_bar.set_postfix({"loss": epoch_loss / (step + 1)})
        epoch_loss_list.append(epoch_loss / (step + 1))

        if (epoch + 1) % val_interval == 0:
            model.eval()
            val_epoch_loss = 0
            for step, batch in enumerate(val_loader):
                images = batch["image"]['data'].squeeze(-1).to(device)
                with torch.no_grad():
                    with autocast(enabled=True):
                        noise = torch.randn_like(images).to(device)
                        timesteps = torch.randint(
                            0, inferer.scheduler.num_train_timesteps, (images.shape[0],), device=images.device
                        ).long()
                        noise_pred = inferer(inputs=images, diffusion_model=model, noise=noise, timesteps=timesteps)
                        val_loss = F.mse_loss(noise_pred.float(), noise.float())

                val_epoch_loss += val_loss.item()
                progress_bar.set_postfix({"val_loss": val_epoch_loss / (step + 1)})
            val_epoch_loss_list.append(val_epoch_loss / (step + 1))

            # Sampling image during training
            noise = torch.randn((1, 1, 64, 64))
            noise = noise.to(device)
            scheduler.set_timesteps(num_inference_steps=total_timesteps)
            with autocast(enabled=True):
                image = inferer.sample(input_noise=noise, diffusion_model=model, scheduler=scheduler)

            plt.figure(figsize=(2, 2))
            plt.imshow(image[0, 0].cpu(), vmin=0, vmax=1, cmap="gray")
            plt.tight_layout()
            plt.axis("off")
            plt.show()

    total_time = time.time() - total_start
    print(f"train completed, total time: {total_time}.")

### Save the trained model

In [ ]:
PATH = os.path.join(os.getcwd(), "checkpoints/checkpoints_T1000_e300")
torch.save(model.state_dict(), PATH)

### Initialize the model

In [ ]:
model = DiffusionModelUNet(
    spatial_dims = 2,
    in_channels = 1,
    out_channels = 1,
    num_channels = (128, 256, 256),
    attention_levels = (False, True, True),
    num_res_blocks = 1,
    num_head_channels = 256,
)
model.to(device)

### Load the trained model

In [ ]:
PATH = os.path.join(os.getcwd(), "checkpoints/checkpoints_T1000_e300")
model.load_state_dict(torch.load(PATH))

## Learning curves

In [ ]:
if not use_pretrained:
    plt.style.use("seaborn-v0_8")
    plt.title("Learning Curves", fontsize=20)
    plt.plot(np.linspace(1, n_epochs, n_epochs), epoch_loss_list, color="C0", linewidth=2.0, label="Train")
    plt.plot(
        np.linspace(val_interval, n_epochs, int(n_epochs / val_interval)),
        val_epoch_loss_list,
        color="C1",
        linewidth=2.0,
        label="Validation",
    )
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    plt.xlabel("Epochs", fontsize=16)
    plt.ylabel("Loss", fontsize=16)
    plt.legend(prop={"size": 14})
    plt.show()

## Plotting sampling process along DDPM's Markov chain

In [ ]:
model.eval()
noise = torch.randn((1, 1, 64, 64))
noise = noise.to(device)
scheduler.set_timesteps(num_inference_steps=total_timesteps)
with autocast(enabled=True):
    image, intermediates = inferer.sample(
        input_noise=noise, diffusion_model=model, scheduler=scheduler, save_intermediates=True, intermediate_steps=100
    )

chain = torch.cat(intermediates, dim=-1)

plt.style.use("default")
plt.imshow(chain[0, 0].cpu(), vmin=0, vmax=1, cmap="gray")
plt.tight_layout()
plt.axis("off")
plt.show()

## Plotting multiple generated images

In [ ]:
n_generations = 16
cols = 4

model.eval()
noise = torch.randn((n_generations, 1, 64, 64))
noise = noise.to(device)
scheduler.set_timesteps(num_inference_steps=total_timesteps)
with autocast(enabled=True):
    image = inferer.sample(
        input_noise=noise, diffusion_model=model, scheduler=scheduler
    )

In [ ]:
image = image.cpu().squeeze()

print(image.shape)
# print(image.min())

plt.figure()
for i in range(n_generations//cols):
    for j in range(cols):
        plt.subplot(n_generations//cols, cols, i*cols+j+1)
        plt.imshow(image[i*cols+j], vmin=0, vmax=1, cmap="gray")
        plt.axis("off")
plt.style.use("default")             
plt.tight_layout()
plt.show()

### Cleanup data directory

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)